In [51]:
import findspark
findspark.init()

In [94]:
import sys
{sys.prefix}

{'C:\\Users\\User\\anaconda3\\envs\\pipeline'}

In [95]:
!conda install -c conda-forge -y --prefix {sys.prefix} requests

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\User\anaconda3\envs\pipeline

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotlipy-0.7.0             |py310he2412df_1003         370 KB  conda-forge
    cffi-1.15.0                |  py310hcbf9ad4_0         231 KB  conda-forge
    charset-normalizer-2.0.12  |     pyhd8ed1ab_0          35 KB  conda-forge
    cryptography-36.0.0        |  py310h21b164f_0         1.0 MB
    idna-3.3                   |     pyhd8ed1ab_0          55 KB  conda-forge
    pycparser-2.21             |     pyhd8ed1ab_0         100 KB  conda-forge
    pyopenssl-22.0.0           |     pyhd8ed1ab_0          49 KB  conda-forge
    pysocks-1.7.1              |  py310h5588dad_4          28 KB  conda-forge
    requests-2.27.1            |     pyhd8ed1ab_0          53 KB  conda-forge


In [139]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField
from pyspark.sql.functions import regexp_extract, lit

In [142]:
import json 
from datetime import datetime
import requests

In [54]:
obs_post_data = pd.read_csv("./관측소 정보.csv")
obs_post_data.head(5)

,Unnamed: 0,obs_post_id,obs_lat,data_type,obs_post_name,obs_lon,obs_object
0,0,DT_0063,35.024178,조위관측소,가덕도,128.810933,"조위,수온,염분,기온,기압,풍속,풍향"
1,1,DT_0032,37.731944,조위관측소,강화대교,126.522222,"조위,기압"
2,2,DT_0031,34.028333,조위관측소,거문도,127.308889,"조위,수온,염분,기온,기압,풍속,풍향"
3,3,DT_0029,34.801389,조위관측소,거제도,128.699167,"조위,수온,염분,기온,기압,풍속,풍향"
4,4,DT_0058,37.560833,조위관측소,경인항,126.601111,"조위,기온,기압,풍속,풍향"


In [63]:
spark = SparkSession.builder.master('local[2]').appName('tempera_obs').getOrCreate()

In [58]:
obs_post_list = obs_post_data['obs_post_id'].values.tolist()
obs_post_list[0:5] # 디버깅 용

['DT_0063', 'DT_0032', 'DT_0031', 'DT_0029', 'DT_0058']

In [64]:
service_key="1LJG14JWnAylzKi5GvmuGg=="
now_datetime=datetime.now().strftime('%Y%m%d')
now_datetime # 디버깅 용

'20220218'

In [46]:
for obs in obs_post_list:
    url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
    response = requests.get(url).json()
    df = spark.createDataFrame(response["result"]["data"])
    df = df.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))

http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0063&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0032&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0031&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0029&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0058&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0026&Date=20220217&ResultType=json
http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0049&Date=20220217&ResultType=json

In [65]:
url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=DT_0063&Date=20220217&ResultType=json"

In [90]:
# response = urllib.request.urlopen(url)
json_str = response.read().decode("utf-8")
# str1 = response.json()
json_object = json.loads(json_str)

In [103]:
response = requests.get(url).json()

In [112]:
type(response)

dict

In [136]:
# response["result"]["data"]
response["result"]["meta"]

{'obs_post_id': 'DT_0063',
 'obs_last_req_cnt': '19984/20000',
 'obs_lat': '35.024178',
 'obs_post_name': '가덕도',
 'obs_lon': '128.810933'}

In [137]:
temp_obs_id=response["result"]["meta"]["obs_post_id"]
temp_obs_id

'DT_0063'

In [128]:
df = spark.createDataFrame(response["result"]["data"])

In [108]:
# rdd = spark.sparkContext.parallelize([response])

# df = spark.read.option("multiLine","true").json(rdd)

In [129]:
type(df)

pyspark.sql.dataframe.DataFrame

In [130]:
df.printSchema()

root
 |-- air_temp: string (nullable = true)
 |-- record_time: string (nullable = true)



In [ ]:
# json_schema = ArrayType(
#     StructType([StructField('air_temp', DoubleType(), nullable=False),
#                 StructField('record_time', StringType(), nullable=False),
#                 StructField('obs_post_name', StringType(), nullable=False),
#                 StructField('obs_lat', StringType(), nullable=False),
#                 StructField('obs_lon', StringType(), nullable=False)]))

In [131]:
df.show(5)

+--------+-------------------+
|air_temp|        record_time|
+--------+-------------------+
|    -1.9|2022-02-17 00:00:00|
|    -2.1|2022-02-17 00:01:00|
|    -1.9|2022-02-17 00:02:00|
|    -2.2|2022-02-17 00:03:00|
|    -2.2|2022-02-17 00:04:00|
+--------+-------------------+
only showing top 5 rows



In [140]:
df = df.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))

In [141]:
df.show(5)

+--------+-------------------+-------+
|air_temp|        record_time|     id|
+--------+-------------------+-------+
|    -1.9|2022-02-17 00:00:00|DT_0063|
|    -2.1|2022-02-17 00:01:00|DT_0063|
|    -1.9|2022-02-17 00:02:00|DT_0063|
|    -2.2|2022-02-17 00:03:00|DT_0063|
|    -2.2|2022-02-17 00:04:00|DT_0063|
+--------+-------------------+-------+
only showing top 5 rows



In [143]:
df.count()

1440